# Partition Overview

In [9]:
 DECLARE @SchemaName nvarchar(100) = 'Sales', @TableName nvarchar(100) = 'Orders_Part'
 DECLARE @ShowPartNr int = 0

 SELECT
    --sc.name + N'.' + so.name as [Schema.Table],
    si.index_id as [Index ID],
    si.type_desc as [Structure],
    si.name as [Index],
    stat.row_count AS [Rows],
    stat.in_row_reserved_page_count * 8./1024./1024. as [In-Row GB],
    stat.lob_reserved_page_count * 8./1024./1024. as [LOB GB],
    p.partition_number AS [Partition #],
	p.data_compression_desc AS [Compression],
    pf.name as [Partition Function],
	lv.value leftValue,
    rv.value rightValue,
    CASE pf.boundary_value_on_right	
        WHEN 1 then 'Right / Lower'
        ELSE 'Left / Upper'
    END as [Boundary Type],
    prv.value as [Boundary Point],
    fg.name as [Filegroup]
FROM sys.partition_functions AS pf
JOIN sys.partition_schemes as ps on ps.function_id=pf.function_id
JOIN sys.indexes as si on si.data_space_id=ps.data_space_id
JOIN sys.objects as so on si.object_id = so.object_id
JOIN sys.schemas as sc on so.schema_id = sc.schema_id
JOIN sys.partitions as p on 
    si.object_id=p.object_id 
    and si.index_id=p.index_id
LEFT JOIN sys.partition_range_values as prv on prv.function_id=pf.function_id
    and p.partition_number= 
        CASE pf.boundary_value_on_right WHEN 1
            THEN prv.boundary_id + 1
        ELSE prv.boundary_id
        END
        /* For left-based functions, partition_number = boundary_id, 
           for right-based functions we need to add 1 */
LEFT JOIN sys.partition_range_values rv ON pf.function_id = rv.function_id
                                           AND p.partition_number = rv.boundary_id
LEFT JOIN sys.partition_range_values lv ON pf.function_id = lv.function_id
                                           AND p.partition_number - 1 = lv.boundary_id
JOIN sys.dm_db_partition_stats as stat on stat.object_id=p.object_id
    and stat.index_id=p.index_id
    and stat.index_id=p.index_id and stat.partition_id=p.partition_id
    and stat.partition_number=p.partition_number
JOIN sys.allocation_units as au on au.container_id = p.hobt_id
    and au.type_desc ='IN_ROW_DATA' 
        /* Avoiding double rows for columnstore indexes. */
        /* We can pick up LOB page count from partition_stats */
JOIN sys.filegroups as fg on fg.data_space_id = au.data_space_id
WHERE sc.name = @SchemaName
AND so.name = @TableName
AND p.partition_number >= @ShowPartNr
ORDER BY [Index ID], [Partition Function], [Partition #];
GO

(89 rows affected)

Total execution time: 00:00:00.255

Index ID Structure Index Rows In-Row GB LOB GB Partition # Compression Partition Function leftValue rightValue Boundary Type Boundary Point Filegroup 1 CLUSTERED IX_Sales_Orders_OrderDate 365 0.00080108593 0.00330352734 1 NONE PF_OrderDate NULL 20181001 Right / Lower NULL USERDATA 1 CLUSTERED IX_Sales_Orders_OrderDate 47 0.00019073437 0.00049591015 2 NONE PF_OrderDate 20181001 20181101 Right / Lower 20181001 USERDATA 1 CLUSTERED IX_Sales_Orders_OrderDate 48 0.00019073437 0.00049591015 3 NONE PF_OrderDate 20181101 20181201 Right / Lower 20181101 USERDATA 1 CLUSTERED IX_Sales_Orders_OrderDate 37 0.00019073437 0.00037383984 4 NONE PF_OrderDate 20181201 20190101 Right / Lower 20181201 USERDATA 1 CLUSTERED IX_Sales_Orders_OrderDate 39 0.00019073437 0.00043487500 5 NONE PF_OrderDate 20190101 20190201 Right / Lower 20190101 USERDATA 1 CLUSTERED IX_Sales_Orders_OrderDate 32 0.00012969921 0.00031280468 6 NONE PF_OrderDate 20190201 20190301 Right / Lower 20190201 USERDATA 1 CLUSTERED IX_Sales_Orders_OrderDate 47 0.00019073437 0.00049591015 7 NONE PF_OrderDate 20190301 20190401 Right / Lower 20190301 USERDATA 1 CLUSTERED IX_Sales_Orders_OrderDate 42 0.00019073437 0.00031280468 8 NONE PF_OrderDate 20190401 20190501 Right / Lower 20190401 USERDATA 1 CLUSTERED IX_Sales_Orders_OrderDate 37 0.00019073437 0.00037383984 9 NONE PF_OrderDate 20190501 20190601 Right / Lower 20190501 USERDATA 1 CLUSTERED IX_Sales_Orders_OrderDate 40 0.00019073437 0.00031280468 10 NONE PF_OrderDate 20190601 20190701 Right / Lower 20190601 USERDATA 1 CLUSTERED IX_Sales_Orders_OrderDate 44 0.00019073437 0.00043487500 11 NONE PF_OrderDate 20190701 20190801 Right / Lower 20190701 USERDATA 1 CLUSTERED IX_Sales_Orders_OrderDate 33 0.00012969921 0.00037383984 12 NONE PF_OrderDate 20190801 20190901 Right / Lower 20190801 USERDATA 1 CLUSTERED IX_Sales_Orders_OrderDate 47 0.00025176953 0.00037383984 13 NONE PF_OrderDate 20190901 20191001 Right / Lower 20190901 USERDATA 1 CLUSTERED IX_Sales_Orders_OrderDate 50 0.00025176953 0.00043487500 14 NONE PF_OrderDate 20191001 20191101 Right / Lower 20191001 USERDATA 1 CLUSTERED IX_Sales_Orders_OrderDate 37 0.00019073437 0.00037383984 15 NONE PF_OrderDate 20191101 20191201 Right / Lower 20191101 USERDATA 1 CLUSTERED IX_Sales_Orders_OrderDate 2 0.00019073437 0.00000000000 16 NONE PF_OrderDate 20191201 20191202 Right / Lower 20191201 PRIMARY 1 CLUSTERED IX_Sales_Orders_OrderDate 3 0.00019073437 0.00006866406 17 NONE PF_OrderDate 20191202 20191203 Right / Lower 20191202 PRIMARY 1 CLUSTERED IX_Sales_Orders_OrderDate 3 0.00006866406 0.00006866406 18 NONE PF_OrderDate 20191203 20191204 Right / Lower 20191203 PRIMARY 1 CLUSTERED IX_Sales_Orders_OrderDate 2 0.00006866406 0.00006866406 19 NONE PF_OrderDate 20191204 20191205 Right / Lower 20191204 PRIMARY 1 CLUSTERED IX_Sales_Orders_OrderDate 1 0.00006866406 0.00006866406 20 NONE PF_OrderDate 20191205 20191206 Right / Lower 20191205 PRIMARY 1 CLUSTERED IX_Sales_Orders_OrderDate 1 0.00006866406 0.00000000000 21 NONE PF_OrderDate 20191206 20191207 Right / Lower 20191206 PRIMARY 1 CLUSTERED IX_Sales_Orders_OrderDate 6 0.00006866406 0.00006866406 22 NONE PF_OrderDate 20191207 20191208 Right / Lower 20191207 PRIMARY 1 CLUSTERED IX_Sales_Orders_OrderDate 1 0.00006866406 0.00006866406 23 NONE PF_OrderDate 20191208 20191209 Right / Lower 20191208 PRIMARY 1 CLUSTERED IX_Sales_Orders_OrderDate 4 0.00006866406 0.00006866406 24 NONE PF_OrderDate 20191209 20191210 Right / Lower 20191209 PRIMARY 1 CLUSTERED IX_Sales_Orders_OrderDate 0 0.00000000000 0.00000000000 25 NONE PF_OrderDate 20191210 20191211 Right / Lower 20191210 PRIMARY 1 CLUSTERED IX_Sales_Orders_OrderDate 2 0.00006866406 0.00006866406 26 NONE PF_OrderDate 20191211 20191212 Right / Lower 20191211 PRIMARY 1 CLUSTERED IX_Sales_Orders_OrderDate 4 0.00006866406 0.00006866406 27 NONE PF_OrderDate 20191212 20191213 Right / Lower 20191212 PRIMARY 1 CLUSTERED IX_Sales_Orders_OrderDate 0 0.00000000000 0.00000000000 28 NONE PF_OrderDate 20191213 20191214 Rig